## TensorFlow Serving Deployment Test
Notebook ini digunakan untuk menguji apakah model TensorFlow Serving 

yang sudah dideploy di Railway bisa menerima request dan mengembalikan hasil prediksi.

In [ ]:
# Import library
import pandas as pd
import tensorflow as tf
import base64
import json
import requests

In [ ]:
# Load dataset Adult Income
data = pd.read_csv("data/adult.csv")
data.head()

In [ ]:
# Ambil input (tanpa kolom target 'income')
inputs = data.drop(columns=["income"]).iloc[0].to_dict()
inputs

In [ ]:
def prepare_json(inputs: dict):
    """
    Convert dictionary input menjadi format TensorFlow Serving JSON
    """
    feature_mapping = {}

    for key, value in inputs.items():
        if isinstance(value, str):
            # string → bytes list
            feature_mapping[key] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=[value.encode()])
            )
        else:
            # numeric → float (lebih aman karena mix int/float)
            feature_mapping[key] = tf.train.Feature(
                float_list=tf.train.FloatList(value=[float(value)])
            )

    # Buat TF Example
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_mapping)
    ).SerializeToString()

    # Encode ke base64
    result = {
        "examples": {
            "b64": base64.b64encode(example).decode()
        }
    }

    return json.dumps({
        "signature_name": "serving_default",
        "instances": [result],
    })

In [ ]:
def predict(inputs):
    json_data = prepare_json(inputs)

    # Ganti sesuai endpoint Railway kamu
    endpoint = "https://<app-name>.up.railway.app/v1/models/income-prediction-model:predict"

    response = requests.post(endpoint, data=json_data)
    result = response.json()

    try:
        prediction = result["predictions"][0][0]
        if prediction > 0.5:
            return ">50K"
        else:
            return "<=50K"
    except:
        return f"Error: {result}"

In [ ]:
# Row pertama
inputs = data.drop(columns=["income"]).iloc[0].to_dict()
print("Prediksi row 0:", predict(inputs))

# Row ke-10
inputs = data.drop(columns=["income"]).iloc[10].to_dict()
print("Prediksi row 10:", predict(inputs))